In [17]:
import random
import statistics

def play_entire_game(num_simulations=10_000_000):
    """
    Simulate the described two-stage game multiple times and
    estimate its expected payout and variance via Monte Carlo.

    Game rules:
      1) Flip a fair coin.
         - If HEADS, flip again:
             => If HEADS again, lose 100 immediately (payout = -100, game ends).
             => If TAILS, proceed to Game1.
         - If TAILS on the first flip, proceed to Game1 directly.

      2) Game1:
         - Roll a fair six-sided die (uniform).
         - Flip a fair coin 4 times to count number of heads (Y).
         - The Game1 payout is X^Y, where X is the die result.

    Returns:
        (estimated_mean, estimated_variance)
    """
    payouts = []

    for _ in range(num_simulations):
        # Stage 1: "Two coin flips" logic
        first_flip = random.choice(['H','T'])
        if first_flip == 'H':
            second_flip = random.choice(['H','T'])
            if second_flip == 'H':
                # Lose 100 and end
                payouts.append(-100)
                continue
        #     else:
        #         # Proceed to Game1
        #         pass
        # else:
        #     # Tails on first flip -> proceed to Game1
        #     pass

        # Now we are in Game1
        # Roll a fair die (1 to 6)
        X = random.randint(1,6)
        # Flip 4 coins, count heads
        Y = sum(random.choice(['H','T']) == 'H' for __ in range(4))
        # Payout = X^Y
        payout = X**Y
        payouts.append(payout)

    # Compute empirical mean and variance
    estimated_mean = statistics.mean(payouts)
    # sample variance (unbiased); for large num_simulations close to true variance
    estimated_variance = statistics.pvariance(payouts)  

    return estimated_mean, estimated_variance

if __name__ == "__main__":
    mean_val, var_val = play_entire_game(num_simulations=1_000_000)
    print(f"Estimated Mean: {mean_val:.4f}")
    print(f"Estimated Variance: {var_val:.4f}")


Estimated Mean: 11.5380
Estimated Variance: 21227.8058


Ehancing this code with vectorization.

In [11]:
import numpy as np
size = 10_000_000
random_values = np.random.uniform(0, 1, size)
result_vector = np.where(random_values < 0.25, 0.25, 0.75)
array = np.full(n, -100)
game1Vector = np.random.randint(1, 7, size) ** np.random.binomial(4, 0.5, size)





NameError: name 'n' is not defined

In [18]:
import numpy as np

size = 10_000_000
# Generate random values and assign 0.25 or 0.75 accordingly
random_values = np.random.uniform(0, 1, size)
result_vector = np.where(random_values < 0.25, 0.25, 0.75)

# Create an array of -100 (make sure it has the same length as result_vector)
array = np.full(size, -100)

# Create the game1Vector as defined
game1Vector = np.random.randint(1, 7, size) ** np.random.binomial(4, 0.5, size)

# Compute the final vector:
# For indices where result_vector is 0.25, multiply by array (-100)
# For indices where result_vector is 0.75, multiply by game1Vector
final_vector = np.where(result_vector == 0.25,
                        array,
                        game1Vector)

# Optionally, print or use final_vector
print(final_vector.size)
print(np.mean(final_vector))
print(np.var(final_vector))

10000000
11.5276305
21312.903623355458


In [14]:
import numpy as np

size = 10_000_000
# Generate random values and assign 0.25 or 0.75 accordingly
random_values = np.random.uniform(0, 1, size)
result_vector = np.where(random_values < 0.25, 0.25, 0.75)

# Create an array of -100 (make sure it has the same length as result_vector)
array = np.full(size, -100)

# Create the game1Vector as defined
game1Vector = np.random.randint(1, 7, size) ** np.random.binomial(4, 0.5, size)

# Split result_vector into two parts based on its values
mask_0_25 = result_vector == 0.25
mask_0_75 = result_vector == 0.75

# Handle 0.25 case: Multiply by array (-100)
final_vector_0_25 = result_vector[mask_0_25] * array[mask_0_25]

# Handle 0.75 case: Multiply by game1Vector
final_vector_0_75 = result_vector[mask_0_75] * game1Vector[mask_0_75]

# Combine the two processed arrays back into one
final_vector = np.empty_like(result_vector)
final_vector[mask_0_25] = final_vector_0_25
final_vector[mask_0_75] = final_vector_0_75

# Optionally, print or use final_vector
print(final_vector.size)
print(np.mean(final_vector))
print(np.var(final_vector))


10000000
21.094293075
10341.667473572254


In [ ]:
import random
import statistics

def play_entire_game(num_simulations=10_000_000):
    """
    Simulate the described two-stage game multiple times and
    estimate its expected payout and variance via Monte Carlo.

    Game rules:
      1) Flip a fair coin.
         - If HEADS, flip again:
             => If HEADS again, lose 100 immediately (payout = -100, game ends).
             => If TAILS, proceed to Game1.
         - If TAILS on the first flip, proceed to Game1 directly.

      2) Game1:
         - Roll a fair six-sided die (uniform).
         - Flip a fair coin 4 times to count number of heads (Y).
         - The Game1 payout is X^Y, where X is the die result.

    Returns:
        (estimated_mean, estimated_variance)
    """
    payouts = []

    for _ in range(num_simulations):
        # Stage 1: "Two coin flips" logic
        first_flip = random.choice(['H','T'])
        if first_flip == 'H':
            second_flip = random.choice(['H','T'])
            if second_flip == 'H':
                # Lose 100 and end
                payouts.append(-100)
                continue
            else:
                # Proceed to Game1
                pass
        else:
            # Tails on first flip -> proceed to Game1
            pass

        # Now we are in Game1
        # Roll a fair die (1 to 6)
        X = random.randint(1,6)
        # Flip 4 coins, count heads
        Y = sum(random.choice(['H','T']) == 'H' for __ in range(4))
        # Payout = X^Y
        payout = X**Y
        payouts.append(payout)

    # Compute empirical mean and variance
    estimated_mean = statistics.mean(payouts)
    # sample variance (unbiased); for large num_simulations close to true variance
    estimated_variance = statistics.pvariance(payouts)  

    return estimated_mean, estimated_variance

if __name__ == "__main__":
    mean_val, var_val = play_entire_game(num_simulations=1_000_000)
    print(f"Estimated Mean: {mean_val:.4f}")
    print(f"Estimated Variance: {var_val:.4f}")
